In [4]:
import boxes
from common import read_int_32, read_int_16, read_int_8, get_variables, indent


# Read data

In [5]:
filename = "13-2014-0830-6260-LA93-0M20-E080.jp2"
data = boxes.read_jp2(filename)
boxes_bytes = boxes.read_boxes(data)
parsed_boxes = boxes.parse_boxes_bytes(boxes_bytes)

In [6]:
data = parsed_boxes[4].data

# Read codestream

In [18]:
# Marker segments

class MarkerSegment(object):
    START_OF_STREAM_MARKER = b'\xff\x4f'
    SIZE_MARKER = b'\xff\x51'
    CODING_STYLE_DEFAULT_MARKER = b'\xff\x52'
    QUANTIZATION_DEFAULT_MARKER = b'\xff\x5c'
    COMMENT_MARKER = b'\xff\x64'
    START_OF_TILEPART_MARKER = b'\xff\x90'
    END_OF_CODESTREAM_MARKER = b'\xff\xd9'
    TILEPART_HEADER_MARKER = b'\xff\x58'
    START_OF_DATA_MARKER = b'\xff\x93'

    @classmethod
    def factory(cls, data, cursor):
        marker = data[cursor:cursor+2]
        if marker == cls.START_OF_STREAM_MARKER:
            return StartOfStreamMarkerSegment(data, cursor)
        elif marker == cls.SIZE_MARKER:
            return SizeMarkerSegment(data, cursor)
        elif marker == cls.CODING_STYLE_DEFAULT_MARKER:
            return CodingStyleDefaultMarkerSegment(data, cursor)
        elif marker == cls.QUANTIZATION_DEFAULT_MARKER:
            return QuantizationDefaultMarkerSegment(data, cursor)
        elif marker == cls.COMMENT_MARKER:
            return CommentMarkerSegment(data, cursor)
        elif marker == cls.START_OF_TILEPART_MARKER:
            return StartOfTilepartMarkerSegment(data, cursor)
        elif marker == cls.END_OF_CODESTREAM_MARKER:
            return EndOfCodestreamMarkerSegment(data, cursor)
        elif marker == cls.TILEPART_HEADER_MARKER:
            return TilepartHeaderMarkerSegment(data, cursor)
        elif marker == cls.START_OF_DATA_MARKER:
            return StartOfDataMarkerSegment(data, cursor)
        else:
            pretty_code = ':'.join("{:02x}".format(x) for x in marker)
            raise NotImplementedError(pretty_code)

    def __repr__(self):
        return "A " + type(self).__name__ + " filled with the data :\n" + '\n'.join(
            [indent("{}: {}".format(k, v)) for k, v in get_variables(self).items()])

    
class StartOfStreamMarkerSegment(MarkerSegment):
    def __init__(self, data, cursor):
        
        
        self.length_in_codestream = 2

    def __repr__(self):
        return "A quiet SOT (start of stream) marker segment"

    
class SizeMarkerSegment(MarkerSegment):
    def __init__(self, data, cursor):
        self.Lsiz_marker_segment_size = read_int_16(data, cursor + 2)
        self.length_in_codestream = self.Lsiz_marker_segment_size + 2
        
        self.Rsiz_capability = read_int_16(data, cursor + 4)
        self.Xsiz_reference_grid_width = read_int_32(data, cursor + 6)
        self.Ysiz_reference_grid_height = read_int_32(data, cursor + 10)
        self.XOsiz_horizontal_offset = read_int_32(data, cursor + 14)
        self.YOsiz_vertical_offset = read_int_32(data, cursor + 18)
        self.XTsiz_tile_width = read_int_32(data, cursor + 22)
        self.YTsiz_tile_height = read_int_32(data, cursor + 26)
        self.XTOsiz_tile_horizontal_offset = read_int_32(data, cursor + 30)
        self.YTOsiz_tile_horizontal_offset = read_int_32(data, cursor + 34)
        
        self.Csiz_nb_components = read_int_16(data, cursor + 38)
        assert self.length_in_codestream == (40 + 3 * self.Csiz_nb_components), "ISO 15444-1 p.26"
        
        components = []
        for i in range(self.Csiz_nb_components):            
            Ssiz_depth = read_int_8(data, cursor + 40 + 3*i)
            XRsiz_horizontal_separation = read_int_8(data, cursor + 41 + 3*i)
            YRsiz_vertical_separation = read_int_8(data, cursor + 42 + 3*i)
            components.append({
                'Ssiz_depth': Ssiz_depth,
                'XRsiz_horizontal_separation': XRsiz_horizontal_separation,
                'YRsiz_vertical_separation': YRsiz_vertical_separation,
                })
        self.components = components

    
class CodingStyleDefaultMarkerSegment(MarkerSegment):
    def __init__(self, data, cursor):
        self.Lcod_marker_segment_size = read_int_16(data, cursor + 2)
        self.length_in_codestream = self.Lcod_marker_segment_size + 2
        
        self.Scod_coding_style = read_int_8(data, cursor + 4)
        self.entropy_code = bool(self.Scod_coding_style & 1)
        self.SOP_segments = bool(self.Scod_coding_style & 2)
        self.EPH_markers = bool(self.Scod_coding_style & 4)
        
        self.progression_order = read_int_8(data, cursor + 5)
        self.nb_layers = read_int_16(data, cursor + 6)
        self.multiple_component_transformation = read_int_8(data, cursor + 8)
        
        self.nb_decomposition_levels = read_int_8(data, cursor + 9)
        self.code_block_width = read_int_8(data, cursor + 10)
        self.code_block_height = read_int_8(data, cursor + 11)
        self.code_block_style = read_int_8(data, cursor + 12)
        self.transformation = read_int_8(data, cursor + 13)
        
        if self.entropy_code:
            assert self.length_in_codestream == 15 + self.nb_decomposition_levels, "ISO 15444-1 p.29"
        else:
            assert self.length_in_codestream == 14
            
        if self.entropy_code:
            precinct_sizes = []
            for i in range(self.nb_decomposition_levels + 1):
                value = read_int_8(data, cursor + 14 + i)
                PPx = value & 15
                PPy = value >> 4
                precinct_sizes.append({
                    'PPx': PPx,
                    'PPy': PPy,
                })
            self.precinct_sizes  = precinct_sizes


class QuantizationDefaultMarkerSegment(MarkerSegment):
    def __init__(self, data, cursor):
        self.Lqcd_marker_segment_size = read_int_16(data, cursor + 2)
        self.length_in_codestream = self.Lqcd_marker_segment_size + 2
        
        self.Sqcd_quantization_style = read_int_8(data, cursor + 4)
        
        self.quantization_style = self.Sqcd_quantization_style & 31
        assert self.quantization_style in {0, 1, 2}, "ISO 15444-1 p.39"
        
        self.nb_guard_bits = self.Sqcd_quantization_style >> 5
        
        self.SPqcd = data[cursor + 5 : cursor + self.length_in_codestream]



class CommentMarkerSegment(MarkerSegment):
    def __init__(self, data, cursor):
        self.Lsot_marker_segment_size = read_int_16(data, cursor + 2)
        self.length_in_codestream = self.Lsot_marker_segment_size + 2
        
        self.Rcom_registration_value = read_int_16(data, cursor + 4)
        
        self.Ccom = data[cursor + 6 : cursor + self.length_in_codestream]


class StartOfTilepartMarkerSegment(MarkerSegment):
    def __init__(self, data, cursor):
        self.Lcom_marker_segment_size = read_int_16(data, cursor + 2)

        self.Isot_tile_index = read_int_16(data, cursor + 4)
        
        self.Psot_total_length = read_int_32(data, cursor + 6)
        if self.Psot_total_length == 0:
            self.Psot_total_length = len(data) - cursor
        self.length_in_codestream = self.Psot_total_length

        self.TPsot_tilepart_index = read_int_8(data, cursor + 10)
        self.TNsot_nb_tilepart = read_int_8(data, cursor + 11)

        tilepart_data = data[cursor + 12 : cursor + self.length_in_codestream]
        
        tilepart_segments = []
        tilepart_cursor = 0
        while True:
            segment = MarkerSegment.factory(tilepart_data, tilepart_cursor)
            tilepart_segments.append(segment)
            tilepart_cursor += segment.length_in_codestream
            if type(segment).__name__ == 'StartOfDataMarkerSegment':
                break
        
        self.segments = tilepart_segments
        self.data = tilepart_data[tilepart_cursor:]
       

    def __repr__(self):
        return "A " + type(self).__name__ + " with data of length {} :\n".format(len(self.data)) + '\n'.join(
            [indent("{}: {}".format(k, v)) for k, v in get_variables(self).items() if k != 'data'])


class EndOfCodestreamMarkerSegment(MarkerSegment):
    def __init__(self, data, cursor):
        self.length_in_codestream = 2

    def __repr__(self):
        return "A quiet end of codestream marker"


class TilepartHeaderMarkerSegment(MarkerSegment):
    def __init__(self, data, cursor):
        self.Lplt_length_marker_segment = read_int_16(data, cursor + 2)
        self.length_in_codestream = self.Lplt_length_marker_segment + 2

        self.Zplt_index = read_int_8(data, cursor + 4)
        
        Iplt_packet_lengths = data[cursor + 5 : self.length_in_codestream]
        lengths = []
        cursor_Iplt = 0
        current_length = 0
        while cursor_Iplt < len(Iplt_packet_lengths):
            length_byte = read_int_8(Iplt_packet_lengths, cursor_Iplt)
            continue_bit = bool(length_byte >> 7)
            value = length_byte & 127
            current_length *= 128
            current_length += value
            if not continue_bit:
                lengths.append(current_length)
                current_length = 0
            cursor_Iplt += 1
        assert current_length == 0
        self.packet_length = lengths

        
class StartOfDataMarkerSegment(MarkerSegment):
    def __init__(self, data, cursor):
        self.length_in_codestream = 2

    def __repr__(self):
        return "A quiet end of codestream marker"


def read_segments(data):
    marker_segments = []
    cursor = 0
    while cursor < len(data):
        marker_segment = MarkerSegment.factory(data, cursor)
        marker_segments.append(marker_segment)
        cursor += marker_segment.length_in_codestream
    return marker_segments



In [19]:
segments = read_segments(data)


In [20]:
for segment in segments:
    print(segment)

A quiet SOT (start of stream) marker segment
A SizeMarkerSegment filled with the data :
    YTOsiz_tile_horizontal_offset: 0
    Csiz_nb_components: 3
    Ysiz_reference_grid_height: 25000
    components: [{'Ssiz_depth': 7, 'YRsiz_vertical_separation': 1, 'XRsiz_horizontal_separation': 1}, {'Ssiz_depth': 7, 'YRsiz_vertical_separation': 1, 'XRsiz_horizontal_separation': 1}, {'Ssiz_depth': 7, 'YRsiz_vertical_separation': 1, 'XRsiz_horizontal_separation': 1}]
    Rsiz_capability: 2
    XTsiz_tile_width: 25000
    Xsiz_reference_grid_width: 25000
    length_in_codestream: 49
    XOsiz_horizontal_offset: 0
    YOsiz_vertical_offset: 0
    XTOsiz_tile_horizontal_offset: 0
    Lsiz_marker_segment_size: 47
    YTsiz_tile_height: 25000
A CodingStyleDefaultMarkerSegment filled with the data :
    Lcod_marker_segment_size: 21
    nb_layers: 12
    Scod_coding_style: 1
    progression_order: 2
    SOP_segments: False
    code_block_height: 4
    code_block_width: 4
    entropy_code: True
    preci